In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
data1=pd.read_csv("/kaggle/input/trust-fraud-detection-in-siot-with-gcn/userid-table 1.csv")
data2=pd.read_csv("/kaggle/input/trust-fraud-detection-in-siot-with-gcn/userid-table 221104.csv")
data3=pd.read_csv("/kaggle/input/trust-fraud-detection-in-siot-with-gcn/users_realtion 3.csv")
# merged_data=pd.read_csv("/kaggle/input/mearged-1-2/merged 1 2.csv")

In [ ]:
data1.isnull().sum()

In [ ]:
data2.isnull().sum()

In [ ]:
data3.isnull().sum()

In [ ]:
data1.sample(5)

In [ ]:
data2.sample(5)

In [ ]:
data3.sample(5)

In [ ]:
!pip install torch

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example tensors
x = torch.rand(5, 5).to(device)
print(x.device)  # Should show 'cuda:0' if GPU is enabled


In [ ]:
!pip install torch-geometric

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

# Function to extract numeric part from ID strings like '1-4'
def extract_numeric_id(id_str):
    # This splits the string like '1-4' into two parts and returns them as integers
    return tuple(map(int, id_str.split('-')))

# Load the datasets
data1 = pd.read_csv("/kaggle/input/trust-fraud-detection-in-siot-with-gcn/userid-table 1.csv")
data2 = pd.read_csv("/kaggle/input/trust-fraud-detection-in-siot-with-gcn/userid-table 221104.csv")
data3 = pd.read_csv("/kaggle/input/trust-fraud-detection-in-siot-with-gcn/users_realtion 3.csv")

# Merge data1 and data2 on 'userid' to create the user feature set
merged_data = pd.merge(data1, data2, on="userid", how="inner")

# Normalize/Transform relevant features
merged_data['lam1000'] = np.log10(merged_data['lam1000'] + 1)
merged_data['lam3000'] = np.log10(merged_data['lam3000'] + 1)
merged_data['lam5000'] = np.log10(merged_data['lam5000'] + 1)

# Extract user features (lam values, efficiency, service request counts)
user_features = merged_data[['userid', 'lam1000', 'lam3000', 'lam5000', 'effencicy_x']].set_index('userid')

# User-to-User edges (based on relation score from data3)
user_edges = data3[['user_x', 'user_y', 'relation']]

# Edge list preparation (user-to-user edges)
edges_user = user_edges[['user_x', 'user_y']].values
relation_scores = user_edges['relation'].values

# Create User-to-Service edges (from serviceid and user interactions)
user_service_edges = []
service_weights = []

# Separate service_id components into two lists
service_ids_part_1 = []
service_ids_part_2 = []

# Iterate through the merged data to create service edges and their weights
for _, row in merged_data.iterrows():
    user_id = row['userid']
    
    # Assuming 'serviceid_y' corresponds to service IDs after merging
    service_id = row['serviceid_y']  
    
    # Extract numeric values from the service ID like '1-4' -> (1, 4)
    service_id_tuple = extract_numeric_id(service_id)
    
    # Append to the edges list (user-service edges)
    user_service_edges.append([user_id, service_id_tuple[0]])  # First part of the service_id
    service_ids_part_1.append(service_id_tuple[0])
    service_ids_part_2.append(service_id_tuple[1])  # Second part of the service_id
    
    # Append corresponding lam values as weights
    service_weights.append([row['lam1000'], row['lam3000'], row['lam5000']])

# Convert to numpy arrays
user_service_edges = np.array(user_service_edges)
service_weights = np.array(service_weights)

# Combining edges for graph construction
edges_combined = np.vstack([edges_user, user_service_edges])

# Ensure edge_weights_combined matches the edge count
# For user-to-user edges, we only have relation scores
# For user-to-service edges, we append lam values as weights
# The service_weights is a 2D array (n_service_edges, 3), and we need to flatten it for combination
edge_weights_combined = np.concatenate([relation_scores, service_weights.reshape(-1)])

# Prepare node features
node_features = user_features.values

# Create edge types for user-to-user (0) and user-to-service (1)
user_user_edges = len(edges_user)  # Number of user-to-user edges
user_service_edges = len(user_service_edges)  # Number of user-to-service edges

# Create edge types: 0 for user-to-user, 1 for user-to-service
edge_types_user_user = np.zeros(user_user_edges, dtype=int)  # Type 0 for user-to-user edges
edge_types_user_service = np.ones(user_service_edges, dtype=int)  # Type 1 for user-to-service edges

# Combine edge types
edge_types_combined = np.concatenate([edge_types_user_user, edge_types_user_service])

# Convert to PyTorch tensors for PyG (PyTorch Geometric)
edge_index = torch.tensor(edges_combined.T, dtype=torch.long)
edge_attr = torch.tensor(edge_weights_combined, dtype=torch.float)
x = torch.tensor(node_features, dtype=torch.float)

# Create the PyG Data object
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, edge_type=torch.tensor(edge_types_combined, dtype=torch.long))

# Print the data object to verify
print(f"Edge Index Shape: {data.edge_index.shape}")
print(f"Edge Type Shape: {data.edge_type.shape}")
print(data)


In [ ]:
print(merged_data.columns)

In [ ]:
# Adjust edge type to match edge index
if len(edge_attr) != edge_index.shape[1]:
    print(f"Adjusting Edge Type from {len(edge_attr)} to {edge_index.shape[1]}.")

    # Ensure edge types match the number of edges
    if len(edge_attr) > edge_index.shape[1]:
        edge_attr = edge_attr[:edge_index.shape[1]]  # Truncate to match
    else:
        raise ValueError(
            f"Edge Type size ({len(edge_attr)}) is smaller than Edge Index size ({edge_index.shape[1]})."
        )

print(f"Adjusted Edge Type Shape: {edge_attr.shape}")


In [ ]:
print(f"Edge Index Shape: {data.edge_index.shape}")  # Should be [2, num_edges]
print(f"Edge Type Shape: {data.edge_type.shape}")        # Should be [num_edges]


In [ ]:
!pip install plotly



In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt
# from torch_geometric.utils import to_networkx

# # Convert the PyTorch Geometric graph to a NetworkX graph
# G = to_networkx(data, to_undirected=True)

# # Create a plot
# plt.figure(figsize=(10, 10))

# # Visualize the graph using NetworkX
# pos = nx.spring_layout(G)  # Use spring layout for better node spacing
# nx.draw(G, pos, with_labels=True, node_size=500, node_color='lightblue', font_size=12, font_weight='bold', edge_color='gray')

# # Show the graph
# plt.title('Graph Visualization using NetworkX and Matplotlib')
# plt.show()

import plotly.graph_objects as go
import networkx as nx
from torch_geometric.utils import to_networkx

# Convert the PyTorch Geometric graph to a NetworkX graph
G = to_networkx(data, to_undirected=True)

# Get node positions using a layout (e.g., spring layout)
pos = nx.spring_layout(G)

# Create node trace for Plotly
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]

# Create edge trace for Plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)

# Create Plotly figure
fig = go.Figure()


# Add edges to the figure
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='lightblue'), hoverinfo='none', mode='lines'))

# Add nodes to the figure
fig.add_trace(go.Scatter(x=node_x, y=node_y, mode='markers', hoverinfo='text', marker=dict(color='lightblue', size=10)))

# Set layout for the Plotly figure
fig.update_layout(title='Interactive Graph Visualization using Plotly', showlegend=False, hovermode='closest')
fig.write_html("graph_visualization.html")
# Show the figure
fig.show()



In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv
from torch_geometric.loader import DataLoader

In [ ]:

# Define the RGCN model
class RGCNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_relations):
        super(RGCNModel, self).__init__()
        # RGCN layers
        self.conv1 = RGCNConv(input_dim, hidden_dim, num_relations, num_bases=4)
        self.conv2 = RGCNConv(hidden_dim, output_dim, num_relations, num_bases=4)

    def forward(self, x, edge_index, edge_type):
        # Apply first RGCN layer
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)  # Apply non-linearity
        # Apply second RGCN layer
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)  # Output probabilities for classification


In [ ]:
# Initialize the model
input_dim = data.x.shape[1]         # Number of input features per node
hidden_dim = 64                     # Hidden layer dimension
output_dim = 2                      # Number of output classes (binary classification in this case)
num_relations = len(set(data.edge_attr.numpy()))  # Number of relations from edge attributes

model = RGCNModel(input_dim, hidden_dim, output_dim, num_relations)


In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:

# Ensure the data has target labels (e.g., binary classification)
data.y = torch.randint(0, 2, (data.x.size(0),))  # Random binary labels for each node

In [ ]:
# Prepare data for training
data.train_mask = torch.rand(data.x.size(0)) < 0.8  # Randomly assign 80% for training
data.test_mask = ~data.train_mask                  # Remaining 20% for testing
edge_type = torch.tensor(data.edge_attr.numpy(), dtype=torch.long)

In [ ]:
# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)  # Forward pass
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute loss for training nodes
    loss.backward()  # Backpropagation
    optimizer.step()  # Update parameters
    return loss.item()

In [ ]:
# Testing loop
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index, data.edge_type)  # Forward pass
        pred = out.argmax(dim=1)  # Get predictions
        acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    return acc


In [ ]:
# Train the model
for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0:
        test_acc = test()
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')

print("Training Complete")

In [ ]:
import matplotlib.pyplot as plt

# Arrays to store loss and accuracy values for visualization
losses = []
accuracies = []

# Training and testing loop with logging for visualization
for epoch in range(1, 300):
    # Train the model for one epoch
    loss = train()
    losses.append(loss)  # Store the loss for this epoch
    
    # Every 20 epochs, test the model
    if epoch % 20 == 0:
        test_acc = test()
        accuracies.append(test_acc)  # Store the test accuracy
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')


In [ ]:
# Visualization
# Plot training loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)  # First subplot for training loss
plt.plot(range(1, epoch + 1), losses, label='Training Loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()

# Plot test accuracy
plt.subplot(1, 2, 2)  # Second subplot for test accuracy
plt.plot(range(20, epoch + 1, 20), accuracies, label='Test Accuracy', color='green', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Test Accuracy Over Epochs')
plt.legend()

# Show plots
plt.tight_layout()
plt.show()